# 0 - Initializations

In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time
import csv

from pylab import rcParams
import seaborn as sns
import math
import os
from datetime import datetime,  timedelta

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import GradientBoostingRegressor

from sklearn.datasets import make_regression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import StackingRegressor
 
from matplotlib import pyplot

from solarpy import irradiance_on_plane
from solarpy import solar_panel
from numpy import array

from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.statespace.sarimax import SARIMAX

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras.models import Sequential, model_from_json
from tensorflow.keras.layers import Input, Dense, GRU, Embedding, LSTM, Dropout, Conv1D, MaxPooling1D, Flatten, GlobalMaxPooling1D, RepeatVector, Lambda, TimeDistributed, Embedding, TimeDistributed, BatchNormalization, Reshape, concatenate, Permute
from tensorflow.keras.optimizers import RMSprop, Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard, ReduceLROnPlateau, CSVLogger
from tensorflow.keras.backend import square, mean
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.keras.utils import to_categorical
from keras.utils.vis_utils import plot_model

%run Database.ipynb
%run AuxFunctions.ipynb

plt.rcParams.update({'figure.figsize':(20,7), 'figure.dpi':300})

# 1 - Data Preparation

Cololect building data

In [ ]:
df_pv, df_consumption = collectFromDatabase(False)

Build entire dataset

In [ ]:
dfinal = getData(local = True)

# 2 - Build diferent dataframes

In [ ]:
a1, a2, a3, a4, a5, v1, v2, v3, v4, t5, d6 = getDatasets(dfinal)

# 3.1 - Define the train and validation sets

In [ ]:
model_name = 'Naive-3' 

In [ ]:
dt = a4.copy()
dataset = '2'

final_features = features()
dt = dt[final_features].copy()

In [ ]:
dt_val = v4.copy()

final_features = features()
dt_val = dt_val[final_features].copy()

In [ ]:
train = dt.values
val = dt_val.values

# 3.2 - Build supervised learning sequences

In [ ]:
n_pre = 60
n_post = 15

dX, dY = [], []
for i in range(len(train)-n_pre-n_post):
    dX.append(train[i:i+n_pre])
trainX = np.array(dX)


for i in range(len(train)-n_pre-n_post):

    ar = np.array(([row[12] for row in train[i+n_pre:i+n_pre+n_post]][4], [row[12] for row in train[i+n_pre:i+n_pre+n_post]][9], [row[12] for row in train[i+n_pre:i+n_pre+n_post]][14]))
    B = ar.reshape(-1, len(ar))
    dY.append(B)
    
trainY = np.array(dY)


vX, vY = [], []
for i in range(len(val)-n_pre-n_post):
    vX.append(val[i:i+n_pre])
valX = np.array(vX)

for i in range(len(val)-n_pre-n_post):

    ar = np.array(([row[12] for row in val[i+n_pre:i+n_pre+n_post]][4], [row[12] for row in val[i+n_pre:i+n_pre+n_post]][9], [row[12] for row in val[i+n_pre:i+n_pre+n_post]][14]))
    B = ar.reshape(-1, len(ar))
    vY.append(B)

valY = np.array(vY)

In [ ]:
trainX_original = trainX.copy()
trainY_original = trainY.copy()
valX_original = valX.copy()
valY_original = valY.copy()

In [ ]:
trainY = trainY.reshape(trainY.shape[0], trainY.shape[2])
valY = valY.reshape(valY.shape[0], valY.shape[2])

In [ ]:
trainX.shape, trainY.shape, valX.shape, valY.shape

# 3.3 - Scale the data

In [ ]:
scalersX = {}

for i in range(trainX.shape[2]):
    scalersX[i] = MinMaxScaler(feature_range=(0, 1))
    trainX[:, :, i] = scalersX[i].fit_transform(trainX[:, :, i]) 

for i in range(valX.shape[2]):
    valX[:, :, i] = scalersX[i].transform(valX[:, :, i]) 

In [ ]:
scalerY = MinMaxScaler(feature_range=(0, 1))
trainY = scalerY.fit_transform(trainY) 
valY = scalerY.transform(valY)

# 3.4 - Perform predictions

In [ ]:
predict = Naive_Predict(valX)

In [ ]:
predict = predict.reshape(len(predict), -1)

In [ ]:
predict_original = predict.copy()

In [ ]:
nan_array = np.empty((n_pre - 1))
nan_array.fill(np.nan)
nan_array2 = np.empty(n_post)
nan_array2.fill(np.nan)
ind = np.arange(n_pre + n_post)

In [ ]:
predict = scalerY.inverse_transform(predict)

# 3.5 - Plot the predictions 

Without intervals

In [ ]:
aux = np.full([valX_original.shape[0],1,15], np.nan)

aux2 = np.full([predict.shape[0],1,15], np.nan)

fig, ax = plt.subplots(figsize=(10, 3.5))
for i in range(600, valX.shape[0], valX.shape[0]):

    aux[i, 0, 4] = valY_original[i, 0, 0]
    aux[i, 0, 9] = valY_original[i, 0, 1]
    aux[i, 0, 14] = valY_original[i, 0, 2]
    
    aux2[i, 0, 4] = predict[i, 0]
    aux2[i, 0, 9] = predict[i, 1]
    aux2[i, 0, 14] = predict[i, 2]
    
    forecasts_original = np.concatenate((nan_array, valX_original[i, -1:, 7], aux2[i, 0, :]))
    ground_truth = np.concatenate((nan_array, valX_original[i, -1:, 7], aux[i, 0, :]))
    network_input = np.concatenate((valX_original[i, :, 7], nan_array2))

    plt.xticks(rotation=45)
    
    SMALLER_SIZE = 18
    SMALL_SIZE = 19
    MEDIUM_SIZE = 23
    BIGGER_SIZE = 25

    #plt.('test title', fontsize=BIGGER_SIZE)
    plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
    plt.rc('axes', titlesize=BIGGER_SIZE)    # fontsize of the axes title
    plt.rc('axes', labelsize=SMALL_SIZE)     # fontsize of the x and y labels
    plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
    plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
    plt.rc('legend', fontsize=SMALLER_SIZE)  # legend fontsize
    plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

    
    
    ax.plot(ind, ground_truth, 'r-o', markersize=10, markeredgecolor='black', linewidth=2, label='Av. Power')
    
    ax.plot(ind, forecasts_original, 'go--', markersize=10, linewidth=2, marker='h', markerfacecolor='lightgreen', \
             markeredgewidth=2, label='Forecast')
            
    ax.plot(ind[40:], network_input[40:], '-o', markersize=10, markeredgecolor='black', linewidth=2, label='Av. Power')
    
 
    
    plt.ticklabel_format(axis='y', style='sci', scilimits=(5,5))
    plt.xlabel('Date')
    plt.ylabel('Available Power [W]')
    plt.title('Model 2 - Available Power - Forecast')
    plt.legend(loc='lower left')
    #plt.savefig('Images/' + model_name, bbox_inches = 'tight')
    plt.savefig('Images/' + model_name + '_op2', bbox_inches = 'tight')

# 3.6 - Get arrays with the predictions of only 5, 10 and 5 minutes ahead

Without Intervals

In [ ]:
predicted_normalized = []
original_normalized = []

predicted = []
original = []

aux = np.full([valY_original.shape[0],1,15], np.nan)
aux5 = np.full([valY.shape[0],1,15], np.nan)
aux1 = np.full([predict.shape[0],1,15], np.nan)
aux2 = np.full([predict.shape[0],1,15], np.nan)

for i in range(0, valX.shape[0]):

    aux[i, 0, 4] = valY_original[i, 0, 0]
    aux[i, 0, 9] = valY_original[i, 0, 1]
    aux[i, 0, 14] = valY_original[i, 0, 2]

    aux1[i, 0, 4] = predict_original[i, 0]
    aux1[i, 0, 9] = predict_original[i, 1]
    aux1[i, 0, 14] = predict_original[i, 2]

    aux2[i, 0, 4] = predict[i, 0]
    aux2[i, 0, 9] = predict[i, 1]
    aux2[i, 0, 14] = predict[i, 2]

    aux5[i, 0, 4] = valY[i, 0]
    aux5[i, 0, 9] = valY[i, 1]
    aux5[i, 0, 14] = valY[i, 2]

    forecasts_normalized = np.concatenate((nan_array, valX[i, -1:, 7], aux1[i, 0, :]))
    forecasts = np.concatenate((nan_array, valX_original[i, -1:, 7], aux2[i, 0, :]))

    ground_truth_normalized = np.concatenate((nan_array, valX[i, -1:, 7], aux5[i, 0, :]))
    ground_truth = np.concatenate((nan_array, valX_original[i, -1:, 7], aux[i, 0, :]))

    predicted_normalized.append((forecasts_normalized[n_pre+4], forecasts_normalized[n_pre+9], forecasts_normalized[n_pre+14]))
    predicted.append((forecasts[n_pre+4], forecasts[n_pre+9], forecasts[n_pre+14]))

    original_normalized.append((ground_truth_normalized[n_pre+4], ground_truth_normalized[n_pre+9], ground_truth_normalized[n_pre+14]))
    original.append((ground_truth[n_pre+4], ground_truth[n_pre+9], ground_truth[n_pre+14]))

# 3.7 - Compute errors

In [ ]:
test_rmse_5, test_rmse_10, test_rmse_15,\
test_mse_5, test_mse_10, test_mse_15,\
test_mae_5, test_mae_10, test_mae_15 = printTestErrors(original, predicted)

In [ ]:
test_rmse_5_n, test_rmse_10_n, test_rmse_15_n,\
test_mse_5_n, test_mse_10_n, test_mse_15_n,\
test_mae_5_n, test_mae_10_n, test_mae_15_n = printTestErrorsNormalized(original_normalized, predicted_normalized)

# 3.9 - Save results

In [ ]:
with open('Test_' + dataset + '.csv', 'a', newline='') as file:
    fieldnames = ['Model', 
                  'Test_MSE_n', 'Test_RMSE_n', 'Test_MAE_n',                  
                  'Test_5_RMSE_n', 'Test_10_RMSE_n', 'Test_15_RMSE_n', 
                  'Test_5_MSE_n', 'Test_10_MSE_n', 'Test_15_MSE_n', 
                  'Test_5_MAE_n', 'Test_10_MAE_n', 'Test_15_MAE_n',
                  
                  'Test_5_RMSE', 'Test_10_RMSE', 'Test_15_RMSE', 
                  'Test_5_MSE', 'Test_10_MSE', 'Test_15_MSE', 
                  'Test_5_MAE', 'Test_10_MAE', 'Test_15_MAE']
    
    writer = csv.DictWriter(file, fieldnames=fieldnames)

    writer.writeheader()

with open('Test_' + dataset + '.csv', 'a', newline='') as file:
    fieldnames = ['Model',
                  'Test_MSE_n', 'Test_RMSE_n', 'Test_MAE_n',                  
                  'Test_5_RMSE_n', 'Test_10_RMSE_n', 'Test_15_RMSE_n', 
                  'Test_5_MSE_n', 'Test_10_MSE_n', 'Test_15_MSE_n', 
                  'Test_5_MAE_n', 'Test_10_MAE_n', 'Test_15_MAE_n',
                
                  'Test_5_RMSE', 'Test_10_RMSE', 'Test_15_RMSE', 
                  'Test_5_MSE', 'Test_10_MSE', 'Test_15_MSE', 
                  'Test_5_MAE', 'Test_10_MAE', 'Test_15_MAE']
    
    writer = csv.DictWriter(file, fieldnames=fieldnames)
       
    writer.writerow({'Model': model_name, 
                     
                      
                     
                     'Test_5_RMSE_n': test_rmse_5_n,
                     'Test_10_RMSE_n': test_rmse_10_n,
                     'Test_15_RMSE_n': test_rmse_15_n, 
                     'Test_5_MSE_n': test_mse_5_n,
                     'Test_10_MSE_n': test_mse_10_n,
                     'Test_15_MSE_n': test_mse_15_n, 
                     'Test_5_MAE_n': test_mae_5_n,
                     'Test_10_MAE_n': test_mae_10_n,
                     'Test_15_MAE_n': test_mae_15_n,
                                     
                     'Test_5_RMSE': test_rmse_5,
                     'Test_10_RMSE': test_rmse_10,
                     'Test_15_RMSE': test_rmse_15, 
                     'Test_5_MSE': test_mse_5,
                     'Test_10_MSE': test_mse_10,
                     'Test_15_MSE': test_mse_15, 
                     'Test_5_MAE': test_mae_5,
                     'Test_10_MAE': test_mae_10,
                     'Test_15_MAE': test_mae_15                 
                                        })